# Intro
En este notebook tenemos el procedimiento en python para leer y procesar los archivos con los movimientos de las compras de las acciones.
En la nomenclatura:
- O = operación de apertura o compra
- C = operación de cierre o venta

In [7]:
# Librerías
import os
import numpy as np
import pandas as pd
from io import StringIO
from dotenv import load_dotenv

# Gráficas
import seaborn as sns

#fecha
from datetime import datetime

# Funciones

In [8]:
# Leemos lo que nos interesa de los csvs
def read_csv(ruta:str, year:int) -> pd.DataFrame:

    # Creamos el path al archivo de origen
    path = f'{ruta}/informe_{str(year)}/informe_csv_{str(year)}.csv'
    # Leer todo como texto plano
    with open(path, encoding='utf-8') as f:
        lines = f.readlines()
    
    # Sacamos el indice de las cabeceras con los nombres de las columnas
    section_indices = [i for i, line in enumerate(lines) if "Operaciones" in line and "Header" in line]

    # Parsear tabla de "Operaciones"
    start = section_indices[0]
    end = section_indices[1] if len(section_indices) > 1 else len(lines)

    trade_lines = lines[start:end]

    df_op = pd.read_csv(StringIO(''.join(trade_lines)))

    return df_op

# Leemos la data

In [9]:
# Cargar el archivo .env
load_dotenv()

# Acceder a las variables
ruta_origen = os.getenv("ORIGEN_PATH")
ruta_destino = os.getenv("DEST_PATH")

# Seleccionamos el año que queremos procesar
year = 2022

In [10]:
# Leemos el df del informe
df = read_csv(ruta=ruta_origen, year=year)

In [11]:
# Leemos los tipos de cambio
path_forex = f'{ruta_origen}/informe_{year}/tipo_cambio_EURUSD_{year}.xls'
forex = pd.read_excel(path_forex)

In [12]:
# forex

# Preprocesamiento

In [13]:
# Quitamos las filas que no nos interesan, que son las de subtotales y totales que guardaremos en otro df
df_ops = df.loc[df['Header'] == 'Data',:]
df_ops_totales = df.loc[df['Header'] != 'Data' ,:]

In [14]:
# Convertimos la columna fecha a un formato estandar. Esto es para luego unir con los tipo de cambio
df_ops['key_forex'] = df_ops.loc[:,'Fecha/Hora'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d, %H:%M:%S') if x != 'No Fecha' else 'No Fecha')

# Convertimos la columna key_forex al formato str '%Y%m%d'
df_ops['key_forex'] = df_ops.loc[:,'key_forex'].apply(lambda x : x.strftime('%Y%m%d') if x != 'No Fecha' else 'No Fecha')

/var/folders/1f/bsxjdbg56jlfz517g824xrh80000gn/T/ipykernel_3678/3307785290.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ops['key_forex'] = df_ops.loc[:,'Fecha/Hora'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d, %H:%M:%S') if x != 'No Fecha' else 'No Fecha')
/var/folders/1f/bsxjdbg56jlfz517g824xrh80000gn/T/ipykernel_3678/3307785290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ops['key_forex'] = df_ops.loc[:,'key_forex'].apply(lambda x : x.strftime('%Y%m%d') if x != 'No Fecha' else 'No

## Preprocesamos el dataset de forex

In [15]:
# Eliminamos las 2 ultimas filas que no nos sirven
forex = forex.iloc[:-2,:]

In [16]:
# Ponemos el formato string en la fecha para que no nos de error al hacer join
forex['key_forex'] = forex.loc[:,'FRECUENCIA'].apply(lambda x : x.strftime('%Y%m%d'))

# Unimos ambos dfs
Unimos los 2 dfs y de esta manera obtenemos los movimientos y los tipos de cambio en cada día y año.

In [17]:
df_supreme = pd.merge(
    left = df_ops,
    right = forex,
    left_on = 'key_forex',
    right_on = 'key_forex',
    how = 'left'
)

In [18]:
# Seleccionamos las columnas que nos interesan tener
columnas = [
    'Símbolo',
    'Fecha/Hora',
    'Cantidad',
    'Precio trans.',
    'Productos',
    'Tarifa/com.',
    'Código',
    'Lunes a Viernes'
]
df_sucio = df_supreme.loc[:, columnas]

In [34]:
# Renombramos algunas columnas

df_rename = df_sucio.rename(columns={
    'Símbolo' : 'Simbolo',
    'Fecha/Hora' : 'Fecha',
    'Precio trans.' : 'Precio_unitario_dolares',
    'Productos' : 'Precio_acciones_dolares',
    'Tarifa/com.' : 'Comision_dolares',
    'Lunes a Viernes' : 'Tipo_cambio',
    'Código' : 'Codigo'
})

In [35]:
# Convertimos la columna de precio en dolares a float
df_rename['Precio_unitario_dolares'] = df_rename.loc[:,'Precio_unitario_dolares'].astype(float)
df_rename['Comision_dolares'] = df_rename.loc[:,'Comision_dolares'].astype(float)
df_rename['Cantidad'] = df_rename.loc[:,'Cantidad'].astype(float)

In [36]:
# Calculamos el precio y la comision en euros unitarios
df_rename['Precio_unitario_euros'] = df_rename.loc[:,'Precio_unitario_dolares'] / df_rename.loc[:,'Tipo_cambio']
df_rename['Comision_euros'] = df_rename.loc[:,'Comision_dolares'] / df_rename.loc[:,'Tipo_cambio']

# Calculamos el precio del paquete de acciones en euros
df_rename['Precio_acciones_euros'] = df_rename.loc[:,'Precio_acciones_dolares'] / df_rename.loc[:,'Tipo_cambio']

In [37]:
# Calculamos los precios junto a las comisiones
df_rename["Precio_euros_con_comisiones"] = df_rename.loc[:,"Precio_acciones_euros"] + df_rename.loc[:,"Comision_euros"]
df_rename["Precio_dolares_con_comisiones"] = df_rename.loc[:,"Precio_acciones_dolares"] + df_rename.loc[:,"Comision_dolares"]

In [38]:
# Sacamos la columna del total de las transacciones
# df_rename['Precio_dolares_total'] = df_rename.loc[:,'Precio_dolares'] * df_rename.loc[:,'Cantidad'] * -1
# df_rename['Precio_euros_total'] = df_rename.loc[:,'Precio_euros'] * df_rename.loc[:,'Cantidad'] * -1

In [39]:
# Obtenemos el primer df limpio con la info desagregada
df_clean_1 = df_rename.copy()

# Df agrupado
En esta sección seguimos los siguientes pasos:
- 1. Dividimos en 2 dfs las compras y las ventas.
- 2. Agrupamos los 2 dfs, de esta manera sabemos el gasto en compras y ventas.
- 3. Obtenemos el precio unitario. La razón está en el último punto de esta sección.
- 4. Unimos los 2 dfs. Obtenemos un df con la información de la accioón en cada fila.
- 5. Calculamos el valor de la compra en función del número de acciones que hayamos vendido, ya que si hay alguna acción que en el ejercicio de 2022 no la hemos vendido, no se usará para calcular el beneficio y pasará al ejercicio del año siguiente.

In [ ]:
#TODO --> cambiar de orde, primero dividir en 2 dfs y luego agruparlos. La razón son las operaciones especiales que tienen un código distinto.

In [40]:
# Agrupamos por el ticker y las compras y las ventas
df_agrup_compras_ventas = df_clean_1.groupby(['Simbolo', 'Codigo']).\
apply(sum, axis=0).\
loc[:,['Cantidad','Precio_dolares_con_comisiones', 'Precio_euros_con_comisiones', 'Comision_dolares', 'Comision_euros']].\
reset_index()

/var/folders/1f/bsxjdbg56jlfz517g824xrh80000gn/T/ipykernel_3678/510993926.py:3: FutureWarning: The provided callable <built-in function sum> is currently using np.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.sum instead.
  apply(sum, axis=0).\
/var/folders/1f/bsxjdbg56jlfz517g824xrh80000gn/T/ipykernel_3678/510993926.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  apply(sum, axis=0).\


In [41]:
# Sacamos el precio por acción. Esto nos ayuda para luego hacer la resta entre las compras y ventas para ver el beneficio.
df_agrup_compras_ventas["Precio_unitario_dolares"] = df_agrup_compras_ventas.loc[:,"Precio_dolares_con_comisiones"] / df_agrup_compras_ventas.loc[:,"Cantidad"] * -1 # Lo ponemos en posi
df_agrup_compras_ventas["Precio_unitario_euros"] = df_agrup_compras_ventas.loc[:,"Precio_euros_con_comisiones"] / df_agrup_compras_ventas.loc[:,"Cantidad"] * -1 # Lo ponemos en posi

In [54]:
# Obtenemos el df de las ventas
df_ventas = df_agrup_compras_ventas.loc[
    df_agrup_compras_ventas["Codigo"].isin(values=["C", "C;P"]),
    ["Simbolo", "Cantidad", "Precio_dolares_con_comisiones", "Precio_euros_con_comisiones"]
]

df_compras = df_agrup_compras_ventas.loc[
    df_agrup_compras_ventas["Codigo"].isin(values=["O", "O;P"]),
    ["Simbolo", "Precio_unitario_dolares", "Precio_unitario_euros"]
]

# Renombramos las columnas de precios unitarios para diferenciar entre compras y ventas
df_ventas = df_ventas.rename(columns={
    "Precio_unitario_dolares" : "Precio_unitario_dolares_ventas",
    "Precio_unitario_euros" : "Precio_unitario_euros_ventas"
})
df_compras = df_compras.rename(columns={
    "Precio_unitario_dolares" : "Precio_unitario_dolares_compras",
    "Precio_unitario_euros" : "Precio_unitario_euros_compras"
})

In [55]:
# Unimos los dfs de compras y ventas
df_hacienda = pd.merge(
    left=df_ventas,
    right=df_compras,
    how="inner",
    on="Simbolo"
)

In [56]:
# Calculamos el precio de las compras
df_hacienda["Precio_dolares_compra"] = df_hacienda.loc[:,"Precio_unitario_dolares_compras"] * df_hacienda.loc[:,"Cantidad"]
df_hacienda["Precio_euros_compra"] = df_hacienda.loc[:,"Precio_unitario_euros_compras"] * df_hacienda.loc[:,"Cantidad"]

In [57]:
df_hacienda["resultado_euros"] = df_hacienda.loc[:,"Precio_euros_compra"] + df_hacienda.loc[:,"Precio_euros_con_comisiones"]
df_hacienda["resultado_dolares"] = df_hacienda.loc[:,"Precio_dolares_compra"] + df_hacienda.loc[:,"Precio_dolares_con_comisiones"]